In [ ]:
import pandas as pd
import numpy as np
datario_tx_precip_alerta_rio = pd.read_csv("database_csv/taxa_precipitacao_alertario.csv")

In [ ]:
datario_tx_precip_alerta_rio.shape

(9710540, 9)

In [ ]:
# Concatenar 'data_particao' e 'horario'
datario_tx_precip_alerta_rio['data_hora'] = datario_tx_precip_alerta_rio['data_particao'] + ' ' + datario_tx_precip_alerta_rio['horario']

# Converter para datetime
datario_tx_precip_alerta_rio['data_hora'] = pd.to_datetime(datario_tx_precip_alerta_rio['data_hora'])
datario_tx_precip_alerta_rio['data_particao'] = pd.to_datetime(datario_tx_precip_alerta_rio['data_particao'])


datario_tx_precip_alerta_rio = datario_tx_precip_alerta_rio.sort_values(by='data_hora', ascending=True)




In [ ]:
num_NaT = datario_tx_precip_alerta_rio['data_hora'].isna().sum()
num_NaT


2761

In [ ]:
datario_tx_precip_alerta_rio = datario_tx_precip_alerta_rio[~pd.isna(datario_tx_precip_alerta_rio["data_hora"])]

In [ ]:
# Preencher valores NaN com 0 nas colunas especificadas
colunas_chuva = ['acumulado_chuva_15_min', 'acumulado_chuva_1_h', 'acumulado_chuva_4_h', 'acumulado_chuva_24_h']

datario_tx_precip_alerta_rio[colunas_chuva] = datario_tx_precip_alerta_rio[colunas_chuva].fillna(0)
datario_tx_precip_alerta_rio[colunas_chuva] = datario_tx_precip_alerta_rio[colunas_chuva].astype(float)

In [ ]:
datario_tx_precip_alerta_rio_red = datario_tx_precip_alerta_rio.loc[datario_tx_precip_alerta_rio['data_particao'].dt.year >= 2016].copy()

datario_tx_precip_alerta_rio_red

,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,horario,data_particao,data_hora
4324741,4_2016-01-01 00:00:00,4,0.0,0.0,0.0,2.4,3.4,00:00:00,2016-01-01,2016-01-01 00:00:00
3013996,32_2016-01-01 00:00:00,32,0.0,0.0,0.0,0.8,2.6,00:00:00,2016-01-01,2016-01-01 00:00:00
1782759,31_2016-01-01 00:00:00,31,0.0,0.0,0.0,13.6,23.8,00:00:00,2016-01-01,2016-01-01 00:00:00
4944657,5_2016-01-01 00:00:00,5,0.0,0.0,0.0,22.2,30.4,00:00:00,2016-01-01,2016-01-01 00:00:00
7887435,33_2016-01-01 00:00:00,33,0.0,0.0,0.0,0.2,3.2,00:00:00,2016-01-01,2016-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...
8012624,7_2024-03-22 14:00:00,7,1.8,9.6,15.8,16.4,16.4,14:00:00,2024-03-22,2024-03-22 14:00:00
2480671,3_2024-03-22 14:00:00,3,3.8,7.2,14.6,23.6,23.6,14:00:00,2024-03-22,2024-03-22 14:00:00
8864160,20_2024-03-22 14:00:00,20,0.2,0.8,6.6,7.2,7.2,14:00:00,2024-03-22,2024-03-22 14:00:00
7139761,25_2024-03-22 14:00:00,25,0.6,2.6,9.8,14.8,14.8,14:00:00,2024-03-22,2024-03-22 14:00:00


In [ ]:
datario_tx_precip_alerta_rio_red.dtypes

primary_key                       object
id_estacao                         int64
acumulado_chuva_15_min           float64
acumulado_chuva_1_h              float64
acumulado_chuva_4_h              float64
acumulado_chuva_24_h             float64
acumulado_chuva_96_h             float64
horario                           object
data_particao             datetime64[ns]
data_hora                 datetime64[ns]
dtype: object

In [ ]:
# # grupo_estacao = datario_tx_precip_alerta_rio_red.groupby('id_estacao')
# datario_tx_precip_alerta_rio_red = datario_tx_precip_alerta_rio_red[datario_tx_precip_alerta_rio_red["id_estacao"] == 1].copy().reset_index(drop = True)
# # Calculando o acumulado de 45 minutos para cada estação
# datario_tx_precip_alerta_rio_red['acumulado_45min'] = datario_tx_precip_alerta_rio_red['acumulado_chuva_15_min'].rolling(window=3).sum()


# datario_tx_precip_alerta_rio_red

In [ ]:
# Definir a função criar_evento_chuvoso
def criar_evento_chuvoso(df):
    # Calculate if each row is the start of an event
    df['evento_inicio'] = (df['acumulado_480min'] != 0) & (df['acumulado_480min'].shift(1) == 0)
    
    # Create a series to represent the event numbers
    evento_nums = df['evento_inicio'].cumsum()
    
    # Create the event IDs
    df['evento_chuvoso'] = df['id_estacao'].astype(str) + '_evento_' + evento_nums.map(str)
    
    # Filter out non-event rows
    df['evento_chuvoso'] = np.where(df['acumulado_480min'] == 0, np.nan, df['evento_chuvoso'])
    
    # Drop intermediate columns
    df.drop(columns=['evento_inicio'], inplace=True)
    
    return df

In [ ]:
# Lista para armazenar os DataFrames modificados para cada estação
dfs_rol_sum = []
estacoes = datario_tx_precip_alerta_rio["id_estacao"].unique().tolist()

# Loop sobre cada id_estacao único no DataFrame original
for estacao in estacoes:
    # Filtrar o DataFrame para a estação atual
    df_estacao = datario_tx_precip_alerta_rio_red[datario_tx_precip_alerta_rio_red['id_estacao'] == estacao].copy()
    
    # Lista para armazenar os dicionários de mapeamento para cada coluna
    mapping_dicts = []
    
    # Loop sobre cada coluna de acumulado de chuva com diferentes intervalos de tempo
    for intervalo in [30, 45, 90, 120, 360, 480, 720]:
        ## Criando o loop para as colunas de minutos

        # Criar a janela para a soma móvel
        window_num = int(intervalo/15)

        # Aplicar a operação de rolling sum na coluna atual
        df_estacao[f'acumulado_{intervalo}min'] = df_estacao["acumulado_chuva_15_min"].rolling(window=window_num).sum()
        
        # Mapear os resultados de volta ao DataFrame original usando o primary_key
        mapping_dict = df_estacao.set_index('primary_key')[f'acumulado_{intervalo}min'].to_dict()
        mapping_dicts.append(mapping_dict)

    df_estacao.fillna(0,inplace = True)
    # Adicionar o DataFrame modificado à lista
    dfs_rol_sum.append(df_estacao)


dfs_rol_sum = list(map(criar_evento_chuvoso, dfs_rol_sum))
    
# Concatenar todos os DataFrames modificados de volta em um único DataFrame
datario_tx_precip_alerta_rio_red = pd.concat(dfs_rol_sum)


In [ ]:
datario_tx_precip_alerta_rio_red = datario_tx_precip_alerta_rio_red.sort_values(by='data_hora', ascending=True)
datario_tx_precip_alerta_rio_red

,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,horario,data_particao,data_hora,acumulado_30min,acumulado_45min,acumulado_90min,acumulado_120min,acumulado_360min,acumulado_480min,acumulado_720min
4944657,5_2016-01-01 00:00:00,5,0.0,0.0,0.0,22.2,30.4,00:00:00,2016-01-01,2016-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3013996,32_2016-01-01 00:00:00,32,0.0,0.0,0.0,0.8,2.6,00:00:00,2016-01-01,2016-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7887435,33_2016-01-01 00:00:00,33,0.0,0.0,0.0,0.2,3.2,00:00:00,2016-01-01,2016-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8534335,2_2016-01-01 00:00:00,2,0.0,0.0,0.0,8.2,18.0,00:00:00,2016-01-01,2016-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1782759,31_2016-01-01 00:00:00,31,0.0,0.0,0.0,13.6,23.8,00:00:00,2016-01-01,2016-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138514,29_2024-03-22 14:00:00,29,0.2,1.8,3.0,20.4,20.4,14:00:00,2024-03-22,2024-03-22 14:00:00,0.8,1.2,2.0,2.2,2.6,2.6,3.4
8864160,20_2024-03-22 14:00:00,20,0.2,0.8,6.6,7.2,7.2,14:00:00,2024-03-22,2024-03-22 14:00:00,0.2,0.4,1.0,1.4,12.6,15.0,16.0
7022422,19_2024-03-22 14:00:00,19,0.0,3.6,17.0,24.2,24.8,14:00:00,2024-03-22,2024-03-22 14:00:00,0.2,0.6,5.2,7.2,41.2,43.4,56.4
2512434,5_2024-03-22 14:00:00,5,1.6,9.6,21.6,26.6,27.4,14:00:00,2024-03-22,2024-03-22 14:00:00,3.8,6.6,16.6,19.4,34.4,38.4,53.6


In [ ]:
datario_tx_precip_alerta_rio_red["data_hora"].unique()

array(['2016-01-01T00:00:00.000000000', '2016-01-01T00:15:00.000000000',
       '2016-01-01T00:30:00.000000000', ...,
       '2024-03-22T13:50:00.000000000', '2024-03-22T13:55:00.000000000',
       '2024-03-22T14:00:00.000000000'], dtype='datetime64[ns]')

In [ ]:
datario_tx_precip_alerta_rio_red.dropna()

,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,horario,data_particao,data_hora,acumulado_30min,acumulado_45min,acumulado_90min,acumulado_120min,acumulado_360min,acumulado_480min,acumulado_720min
4933371,4_2016-01-01 11:45:00,4,0.0,0.0,0.0,2.4,3.4,11:45:00,2016-01-01,2016-01-01 11:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
590342,32_2016-01-01 11:45:00,32,0.0,0.0,0.0,0.8,2.6,11:45:00,2016-01-01,2016-01-01 11:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1209168,33_2016-01-01 11:45:00,33,0.0,0.0,0.0,0.2,3.2,11:45:00,2016-01-01,2016-01-01 11:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5560033,5_2016-01-01 11:45:00,5,0.0,0.0,0.0,22.2,30.4,11:45:00,2016-01-01,2016-01-01 11:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3675226,2_2016-01-01 11:45:00,2,0.0,0.0,0.0,8.2,18.0,11:45:00,2016-01-01,2016-01-01 11:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138514,29_2024-03-22 14:00:00,29,0.2,1.8,3.0,20.4,20.4,14:00:00,2024-03-22,2024-03-22 14:00:00,0.8,1.2,2.0,2.2,2.6,2.6,3.4
8864160,20_2024-03-22 14:00:00,20,0.2,0.8,6.6,7.2,7.2,14:00:00,2024-03-22,2024-03-22 14:00:00,0.2,0.4,1.0,1.4,12.6,15.0,16.0
7022422,19_2024-03-22 14:00:00,19,0.0,3.6,17.0,24.2,24.8,14:00:00,2024-03-22,2024-03-22 14:00:00,0.2,0.6,5.2,7.2,41.2,43.4,56.4
2512434,5_2024-03-22 14:00:00,5,1.6,9.6,21.6,26.6,27.4,14:00:00,2024-03-22,2024-03-22 14:00:00,3.8,6.6,16.6,19.4,34.4,38.4,53.6
